# --- 1. Importação das Bibliotecas ---

In [ ]:
import kagglehub
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from torch import Tensor
from scipy.spatial.distance import cosine
from tqdm import tqdm

pd.set_option('display.max_colwidth', None)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# --- 2. Funções Essenciais para Geração de Embeddings ---

In [ ]:
# --- 2. Funções Essenciais para Geração de Embeddings ---

def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    """
    Realiza a média dos embeddings dos tokens para obter um único embedding para a sentença.
    """
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


In [8]:
# --- 3. Download e Carregamento dos Dados ---
print("Baixando o conjunto de dados MedQuad do Kaggle...")
try:
    path = kagglehub.dataset_download("pythonafroz/medquad-medical-question-answer-for-ai-research")
    print(f"Dataset baixado em: {path}")
    df = pd.read_csv(f"{path}/medquad.csv")
    
    # --- ALTERAÇÃO AQUI ---
    # Carregando apenas as 500 primeiras perguntas do conjunto de dados.
    # Usamos .copy() para evitar avisos de SettingWithCopyWarning do pandas.
    df_amostra = df.head(500).copy()
    
    print(f"\nUtilizando as primeiras {len(df_amostra)} perguntas do dataset.")
    print("Exemplo de dados:")
    print(df_amostra[['question', 'answer']].head(3))
except Exception as e:
    print(f"Erro ao baixar ou carregar o dataset: {e}")
    exit()

Baixando o conjunto de dados MedQuad do Kaggle...
Dataset baixado em: /home/codespace/.cache/kagglehub/datasets/pythonafroz/medquad-medical-question-answer-for-ai-research/versions/1

Utilizando as primeiras 500 perguntas do dataset.
Exemplo de dados:
                              question  \
0             What is (are) Glaucoma ?   
1               What causes Glaucoma ?   
2  What are the symptoms of Glaucoma ?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [9]:
# --- 4. Carregamento do Modelo e Tokenizador (sem alterações) ---
print("\nCarregando o modelo BERT pré-treinado (bert-base-uncased)...")
nome_modelo = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(nome_modelo)
model = AutoModel.from_pretrained(nome_modelo)
print("Modelo carregado com sucesso.")


Carregando o modelo BERT pré-treinado (bert-base-uncased)...
Modelo carregado com sucesso.


In [10]:
# --- 5. Pré-cálculo dos Embeddings para o Conjunto de Dados (CÓDIGO MODIFICADO) ---
# Esta é a etapa mais demorada, mas só precisa ser executada uma vez.
print("\nIniciando a geração de embeddings para as perguntas do nosso conjunto de dados...")
print("Este processo será mais rápido pois estamos usando apenas 500 perguntas.")

# Lista para armazenar os embeddings pré-calculados
embeddings_dataset = []

# --- ALTERAÇÃO AQUI ---
# Imprimindo as 10 primeiras perguntas para identificação
print("\nAs 10 primeiras perguntas que serão processadas:")

# Usamos o tqdm para mostrar uma barra de progresso
# Usamos enumerate para ter um contador (index) no loop
for index, pergunta in enumerate(tqdm(df_amostra['question'], desc="Processando perguntas")):
    # Imprime apenas as 10 primeiras
    if index < 10:
        # Truncamos a pergunta em 80 caracteres para não poluir a tela
        print(f"  {index + 1}: {pergunta[:80]}...")
        if index == 9:
             print("  ...") # Indica que a impressão de amostras terminou

    try:
        # Tokeniza a pergunta
        tokens_pergunta = tokenizer(pergunta, return_tensors="pt", padding=True, truncation=True, max_length=512)
        
        # Gera os embeddings dos tokens
        with torch.no_grad():
            outputs = model(**tokens_pergunta)
        
        # Aplica o pooling para obter o embedding da sentença
        embedding = average_pool(outputs.last_hidden_state, tokens_pergunta['attention_mask'])
        
        # Converte para um formato mais simples (lista de floats) e adiciona à nossa lista
        embeddings_dataset.append(embedding.cpu().numpy().flatten())
    except Exception as e:
        print(f"Erro ao processar a pergunta: '{pergunta}'. Erro: {e}")
        # Adiciona um vetor nulo em caso de erro para manter a correspondência de índice
        embeddings_dataset.append(None)

# Adiciona os vetores gerados como uma nova coluna no DataFrame
df_amostra['vetores'] = embeddings_dataset
# Remove linhas onde o embedding falhou
df_amostra.dropna(subset=['vetores'], inplace=True)
print("\nEmbeddings gerados e armazenados com sucesso!")


Iniciando a geração de embeddings para as perguntas do nosso conjunto de dados...
Este processo será mais rápido pois estamos usando apenas 500 perguntas.

As 10 primeiras perguntas que serão processadas:


Processando perguntas:   0%|          | 1/500 [00:00<01:02,  8.05it/s]

  1: What is (are) Glaucoma ?...
  2: What causes Glaucoma ?...


Processando perguntas:   1%|          | 3/500 [00:00<00:46, 10.74it/s]

  3: What are the symptoms of Glaucoma ?...
  4: What are the treatments for Glaucoma ?...
  5: What is (are) Glaucoma ?...


...

  6: What is (are) Glaucoma ?...
  7: What is (are) Glaucoma ?...
  8: Who is at risk for Glaucoma? ?...


Processando perguntas:   2%|▏         | 11/500 [00:00<00:44, 11.05it/s]

  9: How to prevent Glaucoma ?...
  10: What are the symptoms of Glaucoma ?...
  ...


Processando perguntas: 100%|██████████| 500/500 [00:32<00:00, 15.37it/s]


Embeddings gerados e armazenados com sucesso!


In [11]:
# --- 6. Função de Busca Semântica ---
def encontrar_resposta_similar(pergunta_usuario: str):
    """
    Recebe uma pergunta do usuário, gera seu embedding e a compara com todos os
    embeddings pré-calculados do dataset para encontrar a resposta mais similar.
    """
    if df_amostra.empty:
        print("O DataFrame de amostra está vazio. Não é possível buscar.")
        return
        
    # 1. Gerar o embedding para a pergunta do usuário
    tokens_usuario = tokenizer(pergunta_usuario, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs_usuario = model(**tokens_usuario)
    embedding_usuario = average_pool(outputs_usuario.last_hidden_state, tokens_usuario['attention_mask'])
    vetor_usuario = embedding_usuario.cpu().numpy().flatten()
    
    # 2. Calcular a similaridade com todas as perguntas do dataset
    melhor_score = -1
    melhor_indice = -1
    
    for i, vetor_dataset in enumerate(df_amostra['vetores']):
        # A distância de cosseno é 1 - similaridade.
        # Logo, a similaridade é 1 - distância.
        similaridade = 1 - cosine(vetor_usuario, vetor_dataset)
        
        if similaridade > melhor_score:
            melhor_score = similaridade
            melhor_indice = i
            
    # 3. Obter a resposta correspondente ao melhor score
    # Usamos .iloc para acessar a linha pelo índice numérico
    indice_original = df_amostra.index[melhor_indice]
    pergunta_similar = df_amostra.loc[indice_original, 'question']
    resposta_encontrada = df_amostra.loc[indice_original, 'answer']
    
    return pergunta_similar, resposta_encontrada, melhor_score


In [12]:
# --- 7. Loop de Interação com o Usuário ---
if __name__ == "__main__":
    print("\n--- Sistema de Busca Semântica de Perguntas Médicas ---")
    print("Digite sua pergunta em inglês. Digite 'sair' para encerrar.")
    
    while True:
        pergunta_do_usuario = input("\nSua pergunta: ")
        
        if pergunta_do_usuario.lower() in ['sair', 'exit', 'quit']:
            print("Encerrando o programa. Até logo!")
            break
            
        if not pergunta_do_usuario.strip():
            print("Por favor, digite uma pergunta válida.")
            continue
        
        # Realiza a busca
        pergunta_sim, resposta, score = encontrar_resposta_similar(pergunta_do_usuario)
        
        # Exibe os resultados
        print("\n--- Resultado Encontrado ---")
        print(f"**Pergunta mais similar no nosso banco de dados (Score: {score:.4f}):**")
        print(f"-> {pergunta_sim}")
        print("\n**Resposta:**")
        print(resposta)
        print("----------------------------")


--- Sistema de Busca Semântica de Perguntas Médicas ---
Digite sua pergunta em inglês. Digite 'sair' para encerrar.

--- Resultado Encontrado ---
**Pergunta mais similar no nosso banco de dados (Score: 1.0000):**
-> What is (are) Glaucoma ?

**Resposta:**
Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out o